In [ ]:
import sys
import pickle
import glob
import networkx as nx
import pickle
import itertools as it
import os
import shutil

import warnings
warnings.filterwarnings(action='once')

# if '/home/anna/BioSimSpace/python' not in sys.path:
#     sys.path.insert(1, '/home/anna/BioSimSpace/python')
# import BioSimSpace as BSS

# if '/home/anna/Documents/cinnabar' not in sys.path:
#     sys.path.insert(1, '/home/anna/Documents/cinnabar')
# import cinnabar

# print(cinnabar.__file__)

print("adding code to the pythonpath...")
code = '/home/anna/Documents/code/python'
if code not in sys.path:
    sys.path.insert(1, code)
import pipeline

from pipeline import *
from pipeline.analysis import *
from pipeline.prep import *
from pipeline.utils import *

# import BioSimSpace.Sandpit.Exscientia as BSS
# import BioSimSpace as BSS


BSS.__file__

import py3Dmol as _py3Dmol
from rdkit import Chem as _Chem
import tempfile as _tempfile

In [ ]:
def draw_labelled_molecule(lig_0):

    # Create a temporary working directory and store the directory name.
    tmp_dir = _tempfile.TemporaryDirectory()
    work_dir = tmp_dir.name

    BSS.IO.saveMolecules(work_dir + "/molecule0", lig_0, "PDB")
    # Load the molecules into RDKit.
    rdmol0 = _Chem.MolFromPDBFile(work_dir + "/molecule0.pdb",
        sanitize=False, removeHs=False)

        
    # Set grid view properties.
    pixels = 900
    width = pixels / 2
    height = pixels

    # Create the view.
    view = _py3Dmol.view(linked=False, width=width,
            height=height)
    style = {"stick":{"colorscheme":"grayCarbon", "linewidth": 0.1}}

    # Add the molecules to the views.
    view.addModel(_Chem.MolToMolBlock(rdmol0), "mol0")

    # Set the style.
    view.setStyle({"model": 0}, style)

    # Highlight the atoms from the mapping.
    for atom0 in lig_0.getAtoms():
        p = rdmol0.GetConformer().GetAtomPosition(atom0.index())
        # p = rdmol0.GetAtomWithIdx(atom0.index())
        view.addSphere({"center" : {"x" : p.x, "y" : p.y, "z" : p.z},
                        "radius" : 0.5,
                        "color"  : "green", "alpha": 0.8},
                        )
        view.addLabel(f"{atom0.index()}",
                        {"position" : {"x" : p.x, "y" : p.y, "z" : p.z}}
                        )
        
        # Set background colour.
        view.setBackgroundColor("white")

        # Zoom to molecule.
        view.zoomTo()

    return view


def view_mapping(lig_0, lig_1, mapping):
        # Write the molecules to PDB format.
        BSS.IO.saveMolecules("molecule0", lig_0, "PDB")
        BSS.IO.saveMolecules("molecule1", lig_1, "PDB")

        # Load the molecules into RDKit.
        rdmol0 = _Chem.MolFromPDBFile("molecule0.pdb",
        sanitize=False, removeHs=False)
        rdmol1 = _Chem.MolFromPDBFile("molecule1.pdb",
        sanitize=False, removeHs=False)

        # mapping = BSS.Align.matchAtoms(
        #                         lig_0, lig_1
        #                         )    
        
        # Set grid view properties.
        viewer0 = (0, 0)
        pixels = 900
        viewergrid = (2, 1)
        viewer1 = (1, 0)
        width = pixels / 2
        height = pixels

        # Create the view.
        view = _py3Dmol.view(linked=False, width=width,
                height=height, viewergrid=viewergrid)
        style = {"stick":{"colorscheme":"grayCarbon", "linewidth": 0.1}}

        # Add the molecules to the views.
        view.addModel(_Chem.MolToMolBlock(rdmol0), "mol0", viewer=viewer0)
        view.addModel(_Chem.MolToMolBlock(rdmol1), "mol1", viewer=viewer1)

        # Set the style.
        view.setStyle({"model": 0}, style, viewer=viewer0)
        view.setStyle({"model": 0}, style, viewer=viewer1)

        # Highlight the atoms from the mapping.
        for atom0, atom1 in mapping.items():
                p = rdmol0.GetConformer().GetAtomPosition(atom0)
                view.addSphere({"center" : {"x" : p.x, "y" : p.y, "z" : p.z},
                                "radius" : 0.5,
                                "color"  : "green", "alpha": 0.8},
                                viewer=viewer0)
                view.addLabel(f"{atom0} \u2192 {atom1}",
                                {"position" : {"x" : p.x, "y" : p.y, "z" : p.z}},
                                viewer=viewer0)
                p = rdmol1.GetConformer().GetAtomPosition(atom1)
                view.addSphere({"center" : {"x" : p.x, "y" : p.y, "z" : p.z},
                                "radius" : 0.5,
                                "color"  : "green", "alpha": 0.8},
                                viewer=viewer1)
                view.addLabel(f"{atom1} \u2192 {atom0}",
                                {"position" : {"x" : p.x, "y" : p.y, "z" : p.z}},
                                viewer=viewer1)
                
        # Set background colour.
        view.setBackgroundColor("white", viewer=viewer0)
        view.setBackgroundColor("white", viewer=viewer1)

        # Zoom to molecule.
        view.zoomTo(viewer=viewer0)
        view.zoomTo(viewer=viewer1)

        return view

In [ ]:
lig1 = "lig_ejm45"
lig2 = "lig_ejm53"
prot = "tyk2"
prep_dir = f"/home/anna/Documents/benchmark/{prot}_benchmark/prep"
name = "lig"

system_1 = BSS.IO.readMolecules(
    [f"{prep_dir}/{lig1}_{name}_equil_solv.rst7", f"{prep_dir}/{lig1}_{name}_equil_solv.prm7"])
system_2 = BSS.IO.readMolecules(
    [f"{prep_dir}/{lig2}_{name}_equil_solv.rst7", f"{prep_dir}/{lig2}_{name}_equil_solv.prm7"])

lig_0 = merge.extract_ligand(system_1)
lig_1 = merge.extract_ligand(system_2)

kwargs = {
      "prematch":{}, # 30:27, 34:31, 41:38, 4:4
        "allow ring size change":False, 
        "allow ring breaking": True,
        "complete rings only": True,
        "ring matches ring only": True,
        "match chiral tag": False,
        "natch valence":False,
        "maximise bonds": False
        }



In [ ]:
merge.no_perturbing_atoms_average(system_1, system_2, **kwargs)

In [ ]:
l0a, l1a, mapping = pipeline.prep.merge.atom_mappings(lig_0, lig_1, **kwargs)

view = view_mapping(lig_0, lig_1, mapping)
view


In [ ]:
merge_molecule = merge.merge_ligands(lig_0, lig_1, **kwargs)

BSS.Notebook.View(merge_molecule).system()

In [ ]:
# changes to mapping

# del mapping[7]
mapping[37] = 13

In [ ]:
print(mapping)

with open(f"/home/anna/Documents/benchmark/new_files/mapping/{lig1}~{lig2}_mapping_dict.pickle", 'wb') as handle:
    pickle.dump(mapping, handle)


In [ ]:
view = draw_labelled_molecule(lig_0)
view

In [ ]:
view = draw_labelled_molecule(lig_1)
view